# Installation

In Julia GPU packages are easy to install: Just do a `Pkg.add("CUDA")`. The only thing you need, is a functional NVIDIA driver, but you don't need to install the CUDA toolkit

In [ ]:
#using Pkg
#Pkg.add("CUDA")

In [1]:
#import Pkg; 
#Pkg.add("BenchmarkTools")

In [19]:
using CUDA
using LinearAlgebra

In [20]:
using BenchmarkTools

In CUDA.jl check if the package is functional, you can call the `versioninfo()` function. Like `Base.versioninfo()`, this will print some information on the available hardware and loaded libraries:

In [21]:
CUDA.versioninfo()

CUDA runtime 11.8, artifact installation
CUDA driver 11.8
NVIDIA driver 520.61.5

CUDA libraries: 
- CUBLAS: 11.11.3
- CURAND: 10.3.0
- CUFFT: 10.9.0
- CUSOLVER: 11.4.1
- CUSPARSE: 11.7.5
- CUPTI: 2022.3.0 (API 18.0.0)
- NVML: 11.0.0+520.61.5

Julia packages: 
- CUDA: 5.4.3
- CUDA_Driver_jll: 0.9.1+1
- CUDA_Runtime_jll: 0.14.1+0

Toolchain:
- Julia: 1.11.1
- LLVM: 16.0.6

2 devices:
  0: Tesla V100-PCIE-16GB (sm_70, 15.778 GiB / 16.000 GiB available)
  1: Tesla V100-PCIE-16GB (sm_70, 15.778 GiB / 16.000 GiB available)


# Add Two Vectors Example

Let have as an example vectors add. Let assume you have two vectors $\vec{a}$ and $\vec{b}$ and you want to add them. You can do it in in many ways in Julia: 
1. simple for loop in CPU
2. julia add (+) in CPU or in GPU
3. GPU kernel programming in CUDA or kernel abstracton using CUDA as backend

In [22]:
# let define our input a, b vectors, and output c vector in CPU
vector_size = 1024
a = rand(1:4, vector_size)
b = rand(1:4, vector_size)
c = zeros(Int, vector_size)

1024-element Vector{Int64}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0

The simple loop CPU loop to add two vectors

In [23]:
for i in 1:10
    c[i] = a[i] + b[i]
end
c

1024-element Vector{Int64}:
 6
 6
 3
 5
 6
 7
 6
 6
 3
 6
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0

Julia add (+) operation in CPU

In [24]:
c = a + b

1024-element Vector{Int64}:
 6
 6
 3
 5
 6
 7
 6
 6
 3
 6
 ⋮
 4
 5
 7
 6
 5
 7
 3
 6
 4

Great!! 
Let see how to use add (+) in GPU to add two vectors using GPU resources.

In [28]:
# We need first to move a and b vectors to GPU and define new dc empty vector in GPU
da = CuArray(a)
db = CuArray(b)
dc = CUDA.zeros(Int, size(a))

1024-element CuArray{Int64, 1, CUDA.DeviceMemory}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0

We can add `da` vector to `db` vector using `+` operator. Thanks to Julio multiple dispatch feature!!!

In [29]:
dc = da + db

1024-element CuArray{Int64, 1, CUDA.DeviceMemory}:
 6
 6
 3
 5
 6
 7
 6
 6
 3
 6
 ⋮
 4
 5
 7
 6
 5
 7
 3
 6
 4

Let us now learn how to write gpu kernel with `CUDA.jl` in Julia.

In array operations, `CUDA.jl`` can leverage implicit parallelism to automatically execute these operations in parallel on a GPU. However, when using kernels, it is the programmer's responsibility to effectively utilize the available parallel execution resources for the specific operation.

In [30]:
function vadd(c, a, b)
    # obtain thread index which should be map the index of a and b
    i = threadIdx().x
    # Each thread will add its own element to c
    c[i] = a[i] + b[i]
    return
end

vadd (generic function with 1 method)

At a high level, that's pretty easy, you just need to write a scalar function and launch that function in parallel using the `@cuda` macro and its `threads` keyword argument

In [31]:
@cuda threads=length(a) vadd(dc, da, db)
dc

1024-element CuArray{Int64, 1, CUDA.DeviceMemory}:
 6
 6
 3
 5
 6
 7
 6
 6
 3
 6
 ⋮
 4
 5
 7
 6
 5
 7
 3
 6
 4

ok this is great but try to set `vector_size` to 10240. You will notice that CPU simple loop and add (+) operator in the CPU and GPU are working, but your hand written GPU code is not working.

Ouch what is going on here?

GPUs have a limited number of threads they can run on a single streaming multiprocessor (SM), but they also have multiple SMs.

To take advantage of them all, we need to run a kernel with multiple blocks.  

In CUDA.jl, the expression `i = threadIdx().x + (blockIdx().x - 1) * blockDim().x` calculates a unique index for each thread across multiple blocks in a CUDA kernel execution. Here's a breakdown of each component and how they contribute to computing this index:

- `threadIdx().x`: This returns the x-coordinate of the thread within its block. It's the thread's index within the block, starting from 1 (unlike C/C++ CUDA where it starts from 0).

- `blockIdx().x`: This gives the x-coordinate of the block within the grid. It represents the block's index in the grid, also starting from 1.

- `blockDim().x`: This represents the number of threads per block along the x-axis.

The formula `i = threadIdx().x + (blockIdx().x - 1) * blockDim().x` is used to compute a global index for each thread. It positions the threads linearly across all blocks. Here's what each part does:

- `(blockIdx().x - 1) * blockDim().x`: This part calculates the offset to the start of the current block. Subtracting 1 from `blockIdx().x` makes it zero-based, and then it is multiplied by the number of threads in each block `(blockDim().x)`. This gives the index of the first thread in the current block relative to the entire grid.

- `threadIdx().x`: Adding this to the block offset gives the specific thread's index within the whole grid.

It similer if you are working in 2D grids. The formula for 2D grids is `i = threadIdx().y * blockDim().y + threadIdx().y`. Here's what each part does:


In [109]:
#To know number of Threads per block
CUDA.attribute(device(), CUDA.DEVICE_ATTRIBUTE_MAX_THREADS_PER_BLOCK)

1024

In [110]:
function vadd(c, a, b)
    # calculates a unique index for each thread across multiple blocks
    i = threadIdx().x + (blockIdx().x - 1) * blockDim().x
    if i <= length(a)
        c[i] = a[i] + b[i]
    end
    return
end

vadd (generic function with 1 method)

In [111]:
@cuda threads=1024 blocks=cld(length(da),1024) vadd(dc, da, db)
dc

1024-element CuArray{Int64, 1, CUDA.DeviceMemory}:
 2
 4
 4
 3
 3
 3
 7
 2
 6
 4
 ⋮
 5
 3
 8
 4
 5
 5
 5
 7
 8

## Add Matrix Multiplication Example

In [112]:
matrix_size = 2048
A = rand(matrix_size, matrix_size)
B = rand(matrix_size, matrix_size)
C = zeros(matrix_size, matrix_size)

2048×2048 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮              ⋱                      ⋮         
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 

The three nested loops implmentation of matrix multiplication in CPU


In [127]:
for i in 1:matrix_size
    Threads.@threads for j in 1:matrix_size
        C[i, j] = 0
        for k in 1:matrix_size
            C[i, j] += A[i, k] * B[k, j]
        end
    end
end
C

Julia mutiplication (*) operation in CPU

In [113]:
C = A * B

2048×2048 Matrix{Float64}:
 509.089  510.348  502.856  507.543  …  514.401  504.3    503.37   516.114
 519.232  526.566  515.182  514.504     511.492  517.574  519.79   525.397
 513.519  505.171  508.243  507.533     507.379  505.279  503.33   512.835
 518.391  519.596  513.286  515.529     512.007  514.813  515.499  522.976
 511.13   521.708  506.156  511.011     510.027  510.334  521.786  517.15
 510.033  512.843  502.064  509.046  …  507.186  514.462  511.407  507.398
 500.526  502.503  498.32   503.397     501.819  507.197  504.563  509.718
 517.251  524.602  511.128  517.558     521.457  518.142  513.97   521.908
 510.031  517.331  502.657  512.068     512.042  503.297  507.88   508.645
 505.563  508.921  507.215  500.042     508.242  507.166  511.379  515.96
   ⋮                                 ⋱             ⋮               
 511.885  514.335  509.953  511.562     512.467  517.093  511.803  518.788
 511.204  517.509  503.653  507.83   …  504.19   508.313  510.737  510.62
 514.399

In [121]:
@benchmark  A * B

BenchmarkTools.Trial: 104 samples with 1 evaluation.
 Range (min … max):  42.659 ms … 59.537 ms  ┊ GC (min … max): 0.00% … 25.34%
 Time  (median):     48.161 ms              ┊ GC (median):    7.84%
 Time  (mean ± σ):   48.459 ms ±  3.254 ms  ┊ GC (mean ± σ):  7.29% ±  5.01%

                  ▅█▂█▅▆▅▂                                     
  ▄▁▄▄█▅▇▇▇▁▄▁▅▄▁▇████████▅▅▁▄▁▁▅▁▇▄▅▁▁▁▁▄▁▁▁▁▅▄▁▄▁▁▁▄▁▁▄▁▁▁▄ ▄
  42.7 ms         Histogram: frequency by time        59.1 ms <

 Memory estimate: 32.00 MiB, allocs estimate: 3.

Now Let see how to use add (*) in GPU to mutiply two matrices using GPU resources.

In [88]:
# We need first to move A and B matrces to GPU and define new DC empty matrix in GPU
DA = CuArray(A)
DB = CuArray(B)
DC = CUDA.zeros(size(A))

2048×2048 CuArray{Float32, 2, CUDA.DeviceMemory}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮              ⋱                      ⋮         
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0

The same way here we can multiply `DA` matrix by `DB` matrix using `*` operator. Thanks again to Julio multiple dispatch feature!!!

In [89]:
DC = DA * DB

2048×2048 CuArray{Float64, 2, CUDA.DeviceMemory}:
 510.872  507.187  520.179  518.381  …  507.109  510.753  528.952  515.195
 515.141  509.509  518.769  519.114     501.728  512.96   531.92   509.071
 508.376  505.47   517.744  508.491     508.649  503.993  529.625  516.51
 509.12   505.797  512.938  509.629     497.167  500.447  518.173  507.102
 516.88   505.166  517.104  517.06      505.078  515.646  532.486  526.457
 518.367  504.533  518.646  511.652  …  505.951  507.462  525.726  514.54
 517.696  512.686  523.199  513.128     502.774  508.019  523.233  516.018
 528.937  523.459  526.988  523.026     516.537  524.125  535.253  524.216
 525.609  521.547  521.866  522.177     515.77   514.085  534.858  520.169
 516.009  508.626  515.507  513.631     497.013  510.418  526.231  512.995
   ⋮                                 ⋱             ⋮               
 512.155  499.859  509.823  509.83      500.542  504.753  516.202  511.068
 526.594  505.914  522.545  516.125  …  511.52   519.952  5

In [122]:
@benchmark  DA * DB

BenchmarkTools.Trial: 2113 samples with 1 evaluation.
 Range (min … max):  22.177 μs …   2.743 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):      2.728 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):    2.366 ms ± 921.696 μs  ┊ GC (mean ± σ):  0.08% ± 1.42%

                                                             █  
  ▄▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▂
  22.2 μs         Histogram: frequency by time         2.74 ms <

 Memory estimate: 1.22 KiB, allocs estimate: 53.

In [93]:
function MatrixMultiplication!(A,B,C)

    row = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    col = (blockIdx().y - 1) * blockDim().y + threadIdx().y

    sum = zero(eltype(C))

    if row <= size(A, 1) && col < size(B, 2)
        for i = 1:size(A, 2)

            #@inbounds disables bounds checking for array accesses for performance optimization.
            @inbounds sum += A[row, i] * B[i, col]
        end
        C[row, col] = sum
    end

    return
end

MatrixMultiplication! (generic function with 1 method)

In [123]:
@cuda threads=(32, 32) blocks=(matrix_size ÷ 32, matrix_size ÷ 32) MatrixMultiplication!(DA, DB, DC)
DC

2048×2048 CuArray{Float64, 2, CUDA.DeviceMemory}:
 510.872  507.187  520.179  518.381  …  507.109  510.753  528.952  515.195
 515.141  509.509  518.769  519.114     501.728  512.96   531.92   509.071
 508.376  505.47   517.744  508.491     508.649  503.993  529.625  516.51
 509.12   505.797  512.938  509.629     497.167  500.447  518.173  507.102
 516.88   505.166  517.104  517.06      505.078  515.646  532.486  526.457
 518.367  504.533  518.646  511.652  …  505.951  507.462  525.726  514.54
 517.696  512.686  523.199  513.128     502.774  508.019  523.233  516.018
 528.937  523.459  526.988  523.026     516.537  524.125  535.253  524.216
 525.609  521.547  521.866  522.177     515.77   514.085  534.858  520.169
 516.009  508.626  515.507  513.631     497.013  510.418  526.231  512.995
   ⋮                                 ⋱             ⋮               
 512.155  499.859  509.823  509.83      500.542  504.753  516.202  511.068
 526.594  505.914  522.545  516.125  …  511.52   519.952  5

In [125]:
@benchmark CUDA.@sync @cuda threads=(32, 32) blocks=(matrix_size ÷ 32, matrix_size ÷ 32) MatrixMultiplication!(DA, DB, DC)

BenchmarkTools.Trial: 257 samples with 1 evaluation.
 Range (min … max):  19.359 ms … 19.665 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     19.457 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   19.459 ms ± 29.755 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                        ▂▅▂▇▆▆▃ ▁▇▄▄█▄▃                        
  ▃▁▁▁▁▁▁▁▃▁▃▁▁▁▁▁▁▁▃▁▁▅███████▇███████▇▃▄▆▄▃▃▃▁▃▃▁▁▁▁▁▁▃▁▃▃▃ ▃
  19.4 ms         Histogram: frequency by time        19.5 ms <

 Memory estimate: 1.05 KiB, allocs estimate: 42.

Ouch!! why the kernel implmentation is slower than Julia multiplication operator?

The answer is that this is only the naive implementation of matrix multiplication in Julia. The performant implementation relies on tiling, where the matrix is divided into smaller submatrices (tiles) that fit more effectively within the GPU’s memory hierarchy, including shared memory and cache. By processing these tiles independently, the GPU can optimize memory access patterns and minimize data transfer overhead.

In a tiled implementation, each thread block on the GPU handles a specific tile of the output matrix, loading portions of the input tiles into shared memory to reduce the repeated global memory access. This approach enables a higher level of parallelism by allowing multiple tiles to be processed concurrently across the GPU cores.

# Kernel Abstraction

Let us explor the naive matrix multiplication example using Kernel abstraction

In [ ]:
#import Pkg; 
#Pkg.add("KernelAbstractions")

In [2]:
using KernelAbstractions

Please note how to write a kernel in KernelAbstractions.jl. There are minimal changes compared to the vendor package. It efficiently abstracts away the index calculations

In [5]:
@kernel function MatrixMultiplication!(A, B, C)
    #Global index of  each thread across multiple blocks in both x and y dimension of the grid
    row, col = @index(Global, NTuple)

    sum = zero(eltype(C))

    if row <= size(A, 1) && col <= size(B, 2)
	for i = 1:size(A, 2)
	   @inbounds sum += A[row, i] * B[i, col]
	end
	@inbounds C[row, col] = sum
     end
end


In [61]:
Backend =  CUDA.CUDABackend()
matrix_size = 2048
T = Float64
DA = rand!(allocate(Backend, T, matrix_size, matrix_size))
DB = rand!(allocate(Backend, T, matrix_size, matrix_size))
DC = KernelAbstractions.zeros(Backend, T, matrix_size, matrix_size)

workgroupsize = (32, 32)

kernel! = MatrixMultiplication!(Backend, workgroupsize)
kernel!(DA, DB, DC, ndrange=(size(DC)))
KernelAbstractions.synchronize(Backend)

isapprox(DC, DA * DB)

true

In [49]:
@benchmark begin
    kernel!(DA, DB, DC, ndrange=(size(DC)))
    KernelAbstractions.synchronize(Backend)
end


BenchmarkTools.Trial: 270 samples with 1 evaluation.
 Range (min … max):  18.389 ms … 18.726 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     18.517 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   18.502 ms ± 47.934 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

               ▃▁                     ▂▁ █▃▇▄▃▁                
  ▃▁▁▁▁▃▃▅▁▆▅▅▅██▇▄▆█▃▁▃▃▃▁▆▆▁▃▄▃█▇▆▆██████████▅▃▄▄▁▃▁▁▃▃▁▁▁▃ ▃
  18.4 ms         Histogram: frequency by time        18.6 ms <

 Memory estimate: 1.61 KiB, allocs estimate: 58.

# Memory copy with KernelAbstractions

Let have another example to show how to use shared memory in KernelAbstractions. This kernel performs a matrix copy using local memory (also known as shared memory in CUDA), which can significantly speed up the memory access times by reducing global memory bandwidth usage. 

In [54]:
@kernel function lmem_copy_kernel!(output, @Const(input))

	# Gets the global index of the thread in a multidimensional grid, which is used to index into the global input and output arrays.
	I, J= @index(Global, NTuple) 
	# Gets the local index within a thread block or workgroup, useful for indexing into locally shared memory.
	i, j = @index(Local, NTuple) # Local index of thread

	#@groupsize() retrieves the dimensions of the thread block or workgroup. 
	#The @uniform ensures that these values are treated as constants that are the same for all threads.
	N = @uniform @groupsize()[1] # blockDim.x 
	M = @uniform @groupsize()[2] # blockDim.y


	tile = @localmem eltype(output) (N, M) # Allocate local (shared) memory

	#First, data from the global input array is loaded into the shared tile array using local indices.
	@inbounds tile[i, j] = input[I, J]

	#@synchronize ensures that all threads in the workgroup have completed their memory writes to the shared memory before proceeding. 
	#This is crucial to prevent race conditions.
	@synchronize

	#Finally, the data is written back from the shared tile array to the global output array.
	@inbounds output[I, J] = tile[i, j]

end


In [59]:
input = rand!(allocate(Backend, T, matrix_size, matrix_size))
output = KernelAbstractions.zeros(Backend, T, matrix_size, matrix_size)

const lmem_copy! = lmem_copy_kernel!(Backend, workgroupsize)
lmem_copy!(output, input, ndrange=size(input))
KernelAbstractions.synchronize(Backend)

all(input == output)


true